In [2]:
%pip show -p selenium pandas webdriver_manager


Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\user\anaconda3\python.exe -m pip show [options] <package> ...

no such option: -p


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 1. 드라이버 세팅
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 창 안 띄움
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 2. 맥도날드 딜리버리 페이지 접속 (버거/세트 카테고리)
url = "https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId=11"
driver.get(url)
time.sleep(3)  # 페이지 로딩 대기

# 3. 정보 추출
names, prices, calories = [], [], []

products = driver.find_elements(By.CLASS_NAME, "product-card")
for product in products:
    try:
        name = product.find_element(By.CLASS_NAME, "product-title").text.strip()
        price = product.find_element(By.CLASS_NAME, "starting-price").text.strip().replace("₩", "").replace(",", "")
        cal_area = product.find_elements(By.CLASS_NAME, "text-default")
        cal = None
        for c in cal_area:
            if 'kcal' in c.text.lower():
                cal = c.text.lower().replace("kcal", "").strip()
                break

        names.append(name)
        prices.append(int(price))
        calories.append(float(cal) if cal else None)
    except Exception as e:
        print(f"오류 발생: {e}")
        continue

driver.quit()

# 4. 데이터프레임 저장
df = pd.DataFrame({
    'name': names,
    'price': prices,
    'calorie': calories
})

df.to_csv('mcd_menu.csv', index=False, encoding='utf-8-sig')
print("스크래핑 완료! 'mcd_menu.csv'로 저장됨.")


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 1. 셀레니움 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 창 안 띄우고 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 2. 맥도날드 딜리버리 메뉴 페이지 접속 (버거&세트)
url = "https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId=11"
driver.get(url)
time.sleep(3)  # 로딩 대기

# 3. 데이터 수집
names, prices, calories = [], [], []

products = driver.find_elements(By.CLASS_NAME, "product-card")
for product in products:
    try:
        name = product.find_element(By.CLASS_NAME, "product-title").text.strip()
        price_raw = product.find_element(By.CLASS_NAME, "starting-price").text.strip()
        price = int(price_raw.replace("₩", "").replace(",", ""))
        
        # 칼로리 찾기
        cal = None
        cal_spans = product.find_elements(By.CLASS_NAME, "text-default")
        for span in cal_spans:
            if "kcal" in span.text.lower():
                cal = float(span.text.lower().replace("kcal", "").strip())
                break

        names.append(name)
        prices.append(price)
        calories.append(cal)
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        continue

# 4. DataFrame으로 변환 후 저장
df = pd.DataFrame({
    'name': names,
    'price': prices,
    'calorie': calories
})

# 5. CSV 파일로 저장
df.to_csv("mcd_menu.csv", index=False, encoding='utf-8-sig')
print("✅ 스크래핑 완료! 'mcd_menu.csv' 파일로 저장되었습니다.")

# 드라이버 종료
driver.quit()


✅ 스크래핑 완료! 'mcd_menu.csv' 파일로 저장되었습니다.
